In [1]:
from clase_sms_ccd import Clase_SMS
from datetime import datetime, timedelta
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

excel_rutas = "RUTAS.xlsx"
df_rutas = pd.read_excel(excel_rutas)
#fecha_hoy = datetime.today()
#fecha_ayer = fecha_hoy - timedelta(days=1)
#fecha_ayer = fecha_ayer.strftime("%Y%m%d")
#fecha_hoy = datetime.today().strftime("%Y%m%d")
#fecha_hoy_txt = datetime.today().strftime("%d.%m.%Y")
fecha_hoy = "20240419"
fecha_ayer = "20240418"
fecha_hoy_txt = "19.04.2024"
ruta_zfir60 = df_rutas["RUTA"][0]
ruta_modelo = df_rutas["RUTA"][1]
ruta_dacxanalista = df_rutas["RUTA"][2]

reporte = Clase_SMS(fecha_hoy, fecha_ayer, fecha_hoy_txt, ruta_zfir60, ruta_modelo, ruta_dacxanalista)

In [2]:
reporte.actualizar_base_celulares()
reporte.exportar_deudores()

Total socios: 709
Socios con celular: 463
Socios sin celular: 246


In [3]:
reporte.preparar_fbl5n()
reporte.preparar_recaudacion()
reporte.preparar_modelo()
reporte.preparar_zfir60()

RECAUDACION: 463 registros.
RECAUDACION DISPONIBLE: 328 registros.
RECAUDACION SOBREGIRO: 22 registros.
RECAUDACION SIN LINEA: 113 registros.
MODELO: 437 registros.
MODELO DISPONIBLE: 422 registros.
MODELO SOBREGIRO: 4 registros.
MODELO SIN LINEA: 11 registros.
ZFIR60: 279 registros.


In [4]:
reporte.exportar_archivos_txt()

MENSAJES LISTOS!!!

Nivel 1: 279 destinatarios.
LD: 900 destinatarios.


In [ ]:
#---------------------------------------- FIN DEL PROGRAMA ----------------------------------------#

In [ ]:
#---------------------------------------- EXPORTAR TXT ----------------------------------------#

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
fecha_hoy = "20240419"
fecha_ayer = "20240418"
fecha_hoy_txt = fecha_hoy_txt

ruta_zfir60 = ruta_zfir60 + "/ZFIR60_" + fecha_hoy + ".xlsx"
ruta_modelo = ruta_modelo + "/Modelo de Evaluación de Pedidos de Equipos_" + fecha_hoy + ".xlsx"

fbl5n = "./bases/FBL5N.xlsx"
reporte_recaudacion = "./bases/Reporte_Recaudacion_" + fecha_hoy + ".csv"
deudores = "./bases/Deudores.xlsx"

modelo = "./modelo-zfir-recaudacion/Modelo de Evaluación de Pedidos de Equipos_" + fecha_hoy + ".xlsx"
zfir = "./modelo-zfir-recaudacion/Zfir" + fecha_hoy + ".xlsx"
recaudacion = "./modelo-zfir-recaudacion/Recaudacion_" + fecha_hoy + ".xlsx"

ruta_dacxanalista = ruta_dacxanalista + "/Nuevo_DACxANALISTA.xlsx"
ruta_base_celulares = "./bases/Base_Celulares_CCD.xlsx"

ld = "./archivos-txt/LD " + fecha_hoy_txt + ".txt"
nivel_1 = "./archivos-txt/Nivel 1 " + fecha_hoy_txt + ".txt"

In [ ]:
df_zfir60 = pd.read_excel(ruta_zfir60, sheet_name="Sheet1")
df_zfir60 = df_zfir60[["Cliente Pa", "Total Vencida"]]
df_zfir60["Total Vencida"] = df_zfir60["Total Vencida"].clip(lower=0)
df_zfir60 = df_zfir60.groupby("Cliente Pa")["Total Vencida"].sum().reset_index()
df_zfir60.to_excel(zfir, sheet_name="BASE", index=False)
df_zfir60.shape

In [ ]:
df_recaudacion = pd.read_csv(reporte_recaudacion, encoding="latin1")
df_recaudacion = df_recaudacion.drop("USER_ID", axis=1)
df_recaudacion = df_recaudacion[df_recaudacion["FECHA_RECAUDACION"] == int(fecha_ayer)]

In [ ]:
df_fbl5n = pd.read_excel(fbl5n)
df_fbl5n = df_fbl5n[df_fbl5n["Cuenta"].notna()]
df_fbl5n = df_fbl5n[df_fbl5n["ACC"] == "PE07"]
df_fbl5n = df_fbl5n[["Cuenta","Importe en ML"]]
df_fbl5n["Importe en ML"] = df_fbl5n["Importe en ML"] * -1
df_fbl5n = df_fbl5n.groupby("Cuenta")["Importe en ML"].sum().reset_index()

In [ ]:
df_modelo = pd.read_excel(modelo, sheet_name="Base")
columnas_modelo = ["DEUDOR", "NOMBRE", "LINEA DISPONIBLE EQUIPOS RESTANTE"]
df_modelo = df_modelo[columnas_modelo]
df_modelo.shape

In [ ]:
df_fbl5n.set_index("Cuenta", inplace=True)
df_recaudacion["FALTA"] = df_recaudacion["SAP"].map(df_fbl5n["Importe en ML"])
df_recaudacion["FALTA"].fillna(0, inplace=True)
df_fbl5n.reset_index(inplace=True)
df_recaudacion["RESTA"] = df_recaudacion["LIMITE_CREDITICIO"] - df_recaudacion["SALDO_ACTUAL"]
df_recaudacion["TOTAL"] = df_recaudacion["FALTA"] + df_recaudacion["RESTA"]
df_recaudacion.to_excel(recaudacion, sheet_name="BASE", index=False)
df_recaudacion.reset_index(inplace=True, drop=True)
df_recaudacion.shape

In [ ]:
df_dacxanalista = pd.read_excel(ruta_dacxanalista, sheet_name="Base_NUEVA")
df_dacxanalista = df_dacxanalista[["DEUDOR", "TIPO_DAC", "ESTADO"]]
estados = ["OPERATIVO CON MOVIMIENTO", "OPERATIVO SIN MOVIMIENTO"]
lista_tipo_dac_no_validos = ["PVA", "TARJETERO", "RED", "RECARGA FFVV", "PROVEEDOR", 
                            "CROSSBORDER", "CDR", "CACE", "AGENTE TELMEX", 
                            "AGENTE TELMEX  / RED", "DAC RURAL"]
df_dacxanalista = df_dacxanalista[df_dacxanalista["ESTADO"].isin(estados)]
df_dacxanalista = df_dacxanalista[~df_dacxanalista["TIPO_DAC"].isin(lista_tipo_dac_no_validos)]

# Valida que los deudores de la base se encuentren operativos en el dacxanalista
deudores_dacxanalista = df_dacxanalista["DEUDOR"].tolist()
print(len(deudores_dacxanalista))
ruta_base_celulares = "./bases/Base_Celulares_CCD.xlsx"
df_base_celulares = pd.read_excel(ruta_base_celulares)
deudores_base = df_base_celulares["DEUDOR"].tolist()

no_encontrados = []
for deudor in deudores_base:
    if deudor not in deudores_dacxanalista:
        no_encontrados.append(deudor)
print("Deudores no encontrados:",len(no_encontrados), no_encontrados)
print("Deudores encontrados:",len(deudores_base)-len(no_encontrados))

# Valida que los deudores tengan un celular valido
df_base_celulares = df_base_celulares[~df_base_celulares["DEUDOR"].isin(no_encontrados)]
df_base_celulares = df_base_celulares[["DEUDOR", "NOMBRE", "CELULAR"]]
df_base_celulares = df_base_celulares[df_base_celulares["CELULAR"]!=0]
print("Deudores con celular:", df_base_celulares.shape[0])

In [ ]:
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("á", "a")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("é", "e")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("í", "i")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("ó", "o")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("ú", "u")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("Á", "A")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("É", "E")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("Í", "I")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("Ó", "O")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("Ú", "U")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("  ", " ")
df_base_celulares["NOMBRE"] = df_base_celulares["NOMBRE"].str.replace("  ", " ")

In [ ]:
# Nivel 1 (ZFIR60)
# 51,Text,"Estimado Socio: Le informamos que tiene una deuda vencida de S/ , puede pagarla desde la Web y App de bancos principales. Mayor informacion en el Portal de Canales: https://portaldistribuidores.claro.com.pe. Creditos y Cobranzas Distribuidores."

# LDE (MODELO)
# 51,Text,"Estimado Socio , le informamos que su Linea Disponible de Equipos de hoy es S/ , puede realizar pedidos de equipos valorizados a precio prepago hasta este importe, sujeto a evaluacion crediticia. Creditos y Cobranzas le desea un excelente dia."
# 51,Text,"Estimado Socio , le informamos que el dia de hoy cuenta con sobregiro en Linea Disponible para compra de Equipos de S/ . Para mas informacion contacte a su analista. Creditos y Cobranzas le desea un excelente dia."
# 51,Text,"Estimado Socio , le informamos que el dia de hoy no cuenta con Linea Disponible para compra de Equipos. Para mas informacion contacte a su analista. Creditos y Cobranzas le desea un excelente dia."


# Recaudación (RECAUDACION)
# 51,Text,"Estimado Socio , le informamos que su Linea Disponible en RECAUDACION es S/ , puede realizar su pedido hasta este importe. Creditos y Cobranzas le desea un excelente dia."
# 51,Text,"Estimado Socio , le informamos que el dia de hoy cuenta con sobregiro en RECAUDACION de S/ . Para mayor informacion contacte a su analista de C&CD."
# 51,Text,"Estimado Socio , le informamos que el día de hoy no cuenta con Linea Disponible en RECAUDACION. Para mayor informacion contacte a su analista de C&CD."


In [ ]:
df_cruce_zfir60 = df_base_celulares.merge(df_zfir60, left_on="DEUDOR", right_on="Cliente Pa", how="left")
df_cruce_zfir60 = df_cruce_zfir60[["CELULAR", "Total Vencida"]]
print(df_cruce_zfir60.shape)
df_cruce_zfir60 = df_cruce_zfir60[df_cruce_zfir60["Total Vencida"] != 0]
df_cruce_zfir60.to_excel("A.xlsx", index=False)
df_cruce_zfir60["Total Vencida"] = df_cruce_zfir60["Total Vencida"].apply(
    lambda x: "{:,.2f}".format(x).replace(",", "x").replace(".", ",").replace("x", "."))
df_cruce_zfir60.reset_index(drop=True, inplace=True)
print(df_cruce_zfir60.shape)

In [ ]:
#51,Text,"Estimado Socio: Le informamos que tiene una deuda vencida de S/ , puede pagarla desde la Web y App de bancos principales. Mayor informacion en el Portal de Canales: https://portaldistribuidores.claro.com.pe. Creditos y Cobranzas Distribuidores."
df_cruce_zfir60["Texto"] = df_cruce_zfir60.apply(
    lambda row: f'51{row["CELULAR"]},Text,"Estimado Socio: Le informamos que tiene una deuda vencida de S/{row["Total Vencida"]}, puede pagarla desde la Web y App de bancos principales. Mayor informacion en el Portal de Canales: https://portaldistribuidores.claro.com.pe. Creditos y Cobranzas Distribuidores."', axis=1)
print(df_cruce_zfir60["Texto"][0])
lista_nivel1 = df_cruce_zfir60["Texto"].to_list()
print(len(lista_nivel1))
with open(nivel_1, "w") as f:
    for item in lista_nivel1:
        f.write("%s\n" % item)

In [ ]:
df_cruce_recaudacion = df_base_celulares.merge(df_recaudacion, left_on="DEUDOR", right_on="SAP", how="left")
df_cruce_recaudacion = df_cruce_recaudacion[["CELULAR", "NOMBRE", "TOTAL"]]
df_cruce_recaudacion["TOTAL"].fillna(0, inplace=True)
df_cruce_recaudacion = df_cruce_recaudacion.sort_values(by="TOTAL", ascending=True)
print(df_cruce_recaudacion.shape)
df_cruce_recaudacion["TIPO"] = df_cruce_recaudacion["TOTAL"].apply(
    lambda x: "DISPONIBLE" if x > 0 else ("SIN_LINEA" if x == 0 else "SOBREGIRO"))
df_cruce_recaudacion["TOTAL"] = df_cruce_recaudacion["TOTAL"].apply(
    lambda x: "{:,.2f}".format(x).replace(",", "x").replace(".", ",").replace("x", "."))
df_cruce_recaudacion.reset_index(drop=True, inplace=True)
print(df_cruce_recaudacion.shape)
df_cruce_recaudacion.to_excel("XD.xlsx", index=False)

In [ ]:
def generar_texto(row):
    if row["TIPO"] == "DISPONIBLE":
        return f'51{row["CELULAR"]},Text,"Estimado Socio {row["NOMBRE"]}, le informamos que su Linea Disponible en RECAUDACION es S/ {row["TOTAL"]}, puede realizar su pedido hasta este importe. Creditos y Cobranzas le desea un excelente dia."'
    elif row["TIPO"] == "SOBREGIRO":
        return f'51{row["CELULAR"]},Text,"Estimado Socio {row["NOMBRE"]}, le informamos que el dia de hoy cuenta con sobregiro en RECAUDACION de S/ {row["TOTAL"]}. Para mayor informacion contacte a su analista de C&CD."'
    elif row["TIPO"] == "SIN_LINEA":
        return f'51{row["CELULAR"]},Text,"Estimado Socio {row["NOMBRE"]}, le informamos que el día de hoy no cuenta con Linea Disponible en RECAUDACION. Para mayor informacion contacte a su analista de C&CD."'


In [ ]:
df_cruce_recaudacion["TEXTO"] = df_cruce_recaudacion.apply(generar_texto, axis=1)

In [ ]:
a = df_cruce_recaudacion["TEXTO"].to_list()
a

In [ ]:
len(a)

In [ ]:
df_cruce_modelo = df_base_celulares.merge(df_modelo, left_on="DEUDOR", right_on="DEUDOR", how="inner")
df_cruce_modelo = df_cruce_modelo[["CELULAR", "NOMBRE_x", "LINEA DISPONIBLE EQUIPOS RESTANTE"]]
df_cruce_modelo = df_cruce_modelo.rename(columns={"NOMBRE_x": "NOMBRE", "LINEA DISPONIBLE EQUIPOS RESTANTE": "TOTAL"})
print(df_cruce_modelo['TOTAL'].isna().any())
df_cruce_modelo = df_cruce_modelo.sort_values(by="TOTAL", ascending=True)
print(df_cruce_modelo.shape)
df_cruce_modelo["TIPO"] = df_cruce_modelo["TOTAL"].apply(
    lambda x: "DISPONIBLE" if x > 0 else ("SIN_LINEA" if x == 0 else "SOBREGIRO"))
df_cruce_modelo["TOTAL"] = df_cruce_modelo["TOTAL"].apply(
    lambda x: "{:,.2f}".format(x).replace(",", "x").replace(".", ",").replace("x", "."))
df_cruce_modelo.reset_index(drop=True, inplace=True)
print(df_cruce_modelo.shape)

In [ ]:
df_cruce_modelo.head(50)

In [ ]:
def generar_texto(row):
    if row["TIPO"] == "DISPONIBLE":
        return f'51{row["CELULAR"]},Text,"Estimado Socio {row["NOMBRE"]}, le informamos que su Linea Disponible de Equipos de hoy es S/ {row["TOTAL"]}, puede realizar pedidos de equipos valorizados a precio prepago hasta este importe, sujeto a evaluacion crediticia. Creditos y Cobranzas le desea un excelente dia."'
    elif row["TIPO"] == "SOBREGIRO":
        return f'51{row["CELULAR"]},Text,"Estimado Socio {row["NOMBRE"]}, le informamos que el dia de hoy cuenta con sobregiro en Linea Disponible para compra de Equipos de S/ {row["TOTAL"]}. Para mas informacion contacte a su analista. Creditos y Cobranzas le desea un excelente dia."'
    elif row["TIPO"] == "SIN_LINEA":
        return f'51{row["CELULAR"]},Text,"Estimado Socio {row["NOMBRE"]}, le informamos que el dia de hoy no cuenta con Linea Disponible para compra de Equipos. Para mas informacion contacte a su analista. Creditos y Cobranzas le desea un excelente dia."'


In [ ]:
df_cruce_modelo["TEXTO"] = df_cruce_modelo.apply(generar_texto, axis=1)

In [ ]:
a = df_cruce_modelo["TEXTO"].to_list()
a

In [ ]:
with open(nivel_1, "w") as f:
    for item in lista_nivel1:
        f.write("%s\n" % item)

In [ ]:
df_LD = pd.concat([df_cruce_modelo, df_cruce_recaudacion], ignore_index=True)
lista_ld = df_LD["TEXTO"].to_list()
with open(ld, "w") as f:
    for item in lista_ld:
        f.write("%s\n" % item)